In [ ]:
!pip install biopython
!pip install num2words
!pip install contractions

     |████████████████████████████████| 2.3MB 9.1MB/s 
     |████████████████████████████████| 102kB 5.7MB/s 
     |████████████████████████████████| 317kB 7.3MB/s 
     |████████████████████████████████| 245kB 13.3MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81704 sha256=5bc5dbe6075ccf825692046a42a136a108102cfe7c7c93312a918f700ef7ba6a
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
from Bio import Entrez
import pandas as pd
import time
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from Preprocessing import preprocess
import pprint
from tqdm import tqdm_notebook as tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
pp = pprint.PrettyPrinter(indent=4)
Entrez.email = "Enter your email id here"

In [ ]:
def search(query):
    retrived_data = Entrez.esearch(db='pubmed', sort='relevance', retmax='100000',retmode='xml',term=query)
    results = Entrez.read(retrived_data)
    return results['IdList']

In [ ]:
def fetch_details(id_list):
    ids = ','.join(id_list)
    retrived_data = Entrez.efetch(db='pubmed',retmode='xml',id=ids)
    results = Entrez.read(retrived_data)
    return results

In [ ]:
id_list = search('saliva') # Change this to find for other keywords also

**Abstarct Non Empty ignored**

In [ ]:
buffer_size = 10000

end = len(id_list)

dictionary_set = []
for start in range(0,end,buffer_size):
  
  results = fetch_details(id_list[start:start+buffer_size])

  for article in results['PubmedArticle']:
    try:
      abstract_full  = "".join([str(part) for part in article['MedlineCitation']['Article']['Abstract']['AbstractText']])
      title = article['MedlineCitation']['Article']['ArticleTitle']
      language = article['MedlineCitation']['Article']["Language"]
      pmid = int(article['MedlineCitation']['PMID'])
      keywords = [str(keyObject) for keyListObject in article['MedlineCitation']['KeywordList'] for keyObject in keyListObject]
      keywords = " ".join(keywords)
      dictionary_set.append({"PMID":pmid,"Abstract":abstract_full,"Title":title,"Language":language,"Keywords":keywords})
      
    except:
      pass
  time.sleep(1)
dataframe = pd.DataFrame.from_dict(dictionary_set)
dataframe.to_csv("With_Empty_Abstract.csv")

**Empty Abstract extracted**

In [ ]:
buffer_size = 10000

end = len(id_list)

dictionary_set = []
for start in range(0,end,buffer_size):
  
  results = fetch_details(id_list[start:start+buffer_size])

  for article in results['PubmedArticle']:
    try:
      abstract_full  = "".join([str(part) for part in article['MedlineCitation']['Article']['Abstract']['AbstractText']])
    except:
      abstract_full = ""
    title = article['MedlineCitation']['Article']['ArticleTitle']
    language = article['MedlineCitation']['Article']["Language"]
    pmid = int(article['MedlineCitation']['PMID'])
    keywords = [str(keyObject) for keyListObject in article['MedlineCitation']['KeywordList'] for keyObject in keyListObject]
    keywords = " ".join(keywords)
    dictionary_set.append({"PMID":pmid,"Abstract":abstract_full,"Title":title,"Language":language,"Keywords":keywords})    
  time.sleep(1)

In [ ]:
dataframe = pd.DataFrame.from_dict(dictionary_set)
dataframe.to_csv("With_Empty_Abstract.csv")

***Process keyword levereging power of pubmed api***

In [ ]:
keywords = open("Data.txt").read().split("\n")
uuid_combined = set([])

for keyword in keywords:
  keyword = keyword.replace(" + ",") AND (")
  keywords = "(" + keyword + ")"
  # print(keywords)
  id_list = search(keywords)
  uuid_combined = uuid_combined.union(set(id_list))
  time.sleep(1)

In [ ]:
buffer_size = 1000
uuid_combined = list(uuid_combined)
end = len(uuid_combined)

dictionary_set = []
for start in range(0,end,buffer_size):
  
  results = fetch_details(uuid_combined[start:start+buffer_size])

  for article in results['PubmedArticle']:
    try:
      abstract_full  = "".join([str(part) for part in article['MedlineCitation']['Article']['Abstract']['AbstractText']])
    except:
      abstract_full = ""
    title = article['MedlineCitation']['Article']['ArticleTitle']
    language = article['MedlineCitation']['Article']["Language"]
    pmid = int(article['MedlineCitation']['PMID'])
    keywords = [str(keyObject) for keyListObject in article['MedlineCitation']['KeywordList'] for keyObject in keyListObject]

    dictionary_set.append({"PMID":pmid,"Abstract":abstract_full,"Title":title,"Language":language,"Keywords":keywords})
    # print("Abstract : ",abstract_full)
    # print("Title : ",title)
    # print("Language : ",language)
    # print("PMID :",pmid)
    # print("Keywords :",keywords)
    
  time.sleep(1)

In [ ]:
dataframe = pd.DataFrame.from_dict(dictionary_set)
dataframe.to_csv("Keyword_Using_Pubmed.csv")

Natural language processing version

In [ ]:
preprocessing = preprocess() #  Load preprocess class
def wordList(fileData): # Generate list of preprocessed words from file
  finalSet = []
  if fileData != "":
    fileData = preprocessing.removeEmail(fileData) # Remove emails 
    fileData = preprocessing.removeWebsite(fileData) # Remove webiste and decimal numbers 
    fileData = preprocessing.contractionsExpand(fileData) # Expand the contracted word
    fileData = preprocessing.caseChange(fileData) # Change case of file to lower case
    fileData = preprocessing.removePunctuations(fileData) # Remove punctutation 
    wordList = preprocessing.wordSeperator(fileData) # Seperate words by space

    for word in wordList: # For each words in word list
      if word.isdecimal(): # If number is present expand it 
        finalSet += preprocessing.changeNumber(word) # Generate number form of the number and add it to final list 
      else: 
        finalSet.append(preprocessing.lemmatizeWord(word)) # If it is word apply lemmatization
  return finalSet # Return the fnal word list

In [ ]:
keywords = open("Data.txt").read().split("\n")
final_keywords = []

for keyword in keywords:
  keyword = keyword.split(" + ")
  words = []
  for k in keyword:
    words.extend(k.split(" "))
  final_keywords.append(words)

In [ ]:
dataframe = pd.read_csv("With_Empty_Abstract.csv")
dataframe = dataframe.fillna("")
dataframe["Abstract"] = dataframe["Abstract"].apply(lambda x:wordList(x))
dataframe["Title"] = dataframe["Title"].apply(lambda x:wordList(x))
dataframe["All"] = dataframe["Title"] + dataframe["Abstract"]

In [ ]:
dataframe["Score"] = False
for keyword in tqdm(final_keywords):
  temp = dataframe["All"].apply(lambda x: set(wordList(" ".join(keyword))).issubset(set(x)))
  dataframe["Score"] = dataframe["Score"] | temp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
dataframe = dataframe[dataframe["Score"]==True]
dataframe["FinalScore"] = 0
dataframe["Abstract"] = dataframe["Abstract"].apply(lambda x: " ".join(x))
dataframe["Title"] = dataframe["Title"].apply(lambda x: " ".join(x))
dataframe["Keywords"] = dataframe["Keywords"].apply(lambda x: " ".join(wordList(x)))

In [ ]:
vectorize = TfidfVectorizer()

for keyword in tqdm(final_keywords):
  keyword_sentence = [" ".join(wordList(" ".join(keyword)))]
  x_abstract = vectorize.fit_transform(dataframe["Abstract"])
  y_abstract = vectorize.transform(keyword_sentence)
  temp = 0.2*cosine_similarity(x_abstract,y_abstract)
  dataframe["FinalScore"] += temp.ravel()

  vectorize = TfidfVectorizer()
  x_title = vectorize.fit_transform(dataframe["Abstract"])
  y_title = vectorize.transform(keyword_sentence)
  temp = 0.5*cosine_similarity(x_title,y_title)
  dataframe["FinalScore"] += temp.ravel()

  vectorize = TfidfVectorizer()
  x_keyword = vectorize.fit_transform(dataframe["Keywords"])
  y_keyword = vectorize.transform(keyword_sentence)
  temp = 0.3*cosine_similarity(x_keyword,y_keyword)
  dataframe["FinalScore"] += temp.ravel()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
dataframe = dataframe.sort_values(["FinalScore"],ascending=False)
(dataframe.drop(columns = ["All","Score","FinalScore"])).to_csv("Keyword_Using_nlp.csv")